## QuantumFinance - Classificação de Assuntos com BERT (Parte 02)

## 1. Setup Inicial


```python
# Etapa 1 de 10
# Importa bibliotecas essenciais e define o dispositivo (GPU ou CPU)

In [ ]:
import torch
import pandas as pd
import numpy as np
import re
import string
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn import preprocessing
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm



In [ ]:
def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Verifica o dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Executando em:", device)

In [ ]:
# Etapa auxiliar
# Parâmetros ajustáveis para facilitar tuning posterior

MAX_LENGTH = 256       # Tamanho máximo de tokens por entrada
BATCH_SIZE = 32        # Tamanho do batch para treino
LEARNING_RATE = 5e-5   # Taxa de aprendizado
EPOCHS = 3             # Número de épocas de treinamento
DEV_MODE = True        # Usa apenas uma amostra do dataset para acelerar testes

## 2. Carregamento dos Dados

```python
# Etapa 2 de 10
# Carrega o dataset CSV com separador ';' e explora as colunas principais

In [ ]:
# Carrega o dataset CSV com separador ';' e aplica modo de desenvolvimento (10%) se ativado

caminho_arquivo = 'tickets_reclamacoes_classificados.csv'
df = pd.read_csv(caminho_arquivo, sep=';')

# Ativa modo de desenvolvimento com amostragem reduzida
if DEV_MODE:
    df = df.sample(frac=0.1, random_state=42).reset_index(drop=True)
    print(f"[MODO DEV ATIVADO] Usando {len(df)} amostras")

print("Amostra dos dados:")
display(df.head())

print("Colunas disponíveis:", df.columns.tolist())
print("\nDistribuição das categorias:")
print(df['categoria'].value_counts())

## Etapa 3 de 10
- Normaliza os textos 

In [ ]:
institution_names = ['chase', 'bank', 'jp', 'gm', 'financial', 'jpmcb']

def remove_punctuation(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

def normalize_text(text):
    text = text.lower()
    text = re.sub(r'\d+|/', '', text)
    text = re.sub(r'\bx\b|\w*xx+\w*', '', text)
    text = remove_punctuation(text)
    text = re.sub(r'\s+', ' ', text).strip()

    for institution in institution_names:
        text = re.sub(r'\b' + re.escape(institution) + r'\b', '[INST]', text)

    return text

# Cria nova coluna com texto normalizado
df['texto_limpo'] = df['descricao_reclamacao'].apply(normalize_text)

# Visualiza resultado
print("Pré-visualização das colunas originais e normalizadas:")
df[['descricao_reclamacao', 'texto_limpo']].head()

## 4. Tokenização com BERT
```python
# Etapa 4 de 10
# Tokeniza os textos normalizados usando BERT e armazena input_ids e attention_mask

In [ ]:
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Tokenização e vetorização dos textos normalizados
def tokenize_bert(text):
    tokens = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=MAX_LENGTH,
        return_tensors='pt'
    )
    return tokens

# Aplica tokenização e extrai os tensores como listas
encoded = df['texto_limpo'].apply(tokenize_bert)
df['input_ids'] = encoded.apply(lambda x: x['input_ids'].squeeze().tolist())
df['attention_mask'] = encoded.apply(lambda x: x['attention_mask'].squeeze().tolist())

# Visualiza colunas com embeddings
print("Pré-visualização dos textos tokenizados:")
df[['texto_limpo', 'input_ids', 'attention_mask']].head()

## 5. Codificação dos Rótulos

```python
# Etapa 5 de 10
# Codifica os rótulos da coluna 'categoria' em valores numéricos

In [ ]:
label_encoder = preprocessing.LabelEncoder()
df['label'] = label_encoder.fit_transform(df['categoria'])

# Visualiza a correspondência
print("Categorias codificadas:")
display(pd.DataFrame({
    'categoria': label_encoder.classes_,
    'label': list(range(len(label_encoder.classes_)))
}))

## 6. Tensores e Split Treino/Teste

```python
# Etapa 6 de 10
# Cria os tensores (input_ids, attention_mask, label) e divide os dados (75/25 com random_state=42)

In [ ]:
# Converte listas para tensores
input_ids_tensor = torch.tensor(df['input_ids'].tolist())
attention_mask_tensor = torch.tensor(df['attention_mask'].tolist())
labels_tensor = torch.tensor(df['label'].tolist())

# Realiza o split dos índices para manter os dados organizados
train_idx, test_idx = train_test_split(
    range(len(df)),
    test_size=0.25,
    random_state=42,
    stratify=df['label']  # garante distribuição proporcional
)

# Cria conjuntos de treino e teste
train_dataset = TensorDataset(
    input_ids_tensor[train_idx],
    attention_mask_tensor[train_idx],
    labels_tensor[train_idx]
)

test_dataset = TensorDataset(
    input_ids_tensor[test_idx],
    attention_mask_tensor[test_idx],
    labels_tensor[test_idx]
)

print(f"Tamanho do treino: {len(train_dataset)}")
print(f"Tamanho do teste: {len(test_dataset)}")


In [ ]:
# verificar o comprimento dos tokens para ajusta de parametros
#df['comprimento_tokens'] = df['texto_limpo'].apply(lambda x: len(tokenizer.tokenize(x)))
#df['comprimento_tokens'].describe()


## 7. Construção e Treinamento do Modelo BERT

```python
# Etapa 7 de 10
# Instancia o modelo BERT para classificação e realiza o treinamento

In [ ]:
num_labels = len(label_encoder.classes_)
model = BertForSequenceClassification.from_pretrained(
    'neuralmind/bert-base-portuguese-cased',
    num_labels=num_labels
)
model.to(device)

# Otimizador e parâmetros
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
batch_size = BATCH_SIZE
epochs = EPOCHS

# Loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Treinamento
model.train()
for epoch in range(epochs):
    print(f"\nÉpoca {epoch + 1}/{epochs}")
    for batch in tqdm(train_loader):
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Loss final da época {epoch + 1}: {loss.item():.4f}")

## 8. Avaliação do Modelo

```python
# Etapa 8 de 10
# Avalia o modelo no conjunto de teste usando F1 Score e classification report

In [ ]:
from sklearn.metrics import classification_report, f1_score

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in DataLoader(test_dataset, batch_size=BATCH_SIZE):
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Relatório de classificação
print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))

# F1 Score Macro
f1 = f1_score(all_labels, all_preds, average='macro')
print(f"F1 Score (macro): {f1:.4f}")

## 9. Salvamento do Modelo

```python
# Etapa 9 de 10
# Salva o modelo treinado, o tokenizer e o codificador de rótulos (label encoder)

In [ ]:
import joblib

# Cria pasta se não existir
import os
os.makedirs("modelo_quantumfinance", exist_ok=True)

# Salva modelo e tokenizer
model.save_pretrained("modelo_quantumfinance")
tokenizer.save_pretrained("modelo_quantumfinance")

# Salva o label encoder
joblib.dump(label_encoder, "modelo_quantumfinance/label_encoder.pkl")

print("Modelo, tokenizer e label encoder salvos em 'modelo_quantumfinance/'")

## 10. Predição com Texto Novo

```python
# Etapa 10 de 10
# Usa o modelo treinado para prever a categoria de um novo texto

In [ ]:
def prever_texto(texto):
    texto_limpo = normalize_text(texto)
    tokens = tokenizer(texto_limpo, return_tensors='pt', truncation=True, padding='max_length', max_length=MAX_LENGTH)
    tokens = {k: v.to(device) for k, v in tokens.items()}
    
    with torch.no_grad():
        output = model(**tokens)
        pred = torch.argmax(output.logits, dim=1).cpu().item()

    return label_encoder.inverse_transform([pred])[0]

# Exemplo de uso:
texto_exemplo = "Gostaria de abrir um banco no brasil."
print("Assunto previsto:", prever_texto(texto_exemplo))

In [ ]:
casos_teste = {
    "caso_cartao_1": "Recebi uma cobrança indevida no meu cartão de crédito e não consigo cancelar.",
    "caso_cartao_2": "Preciso desbloquear meu cartão pré-pago que foi bloqueado sem explicação.",
    "caso_emprestimo": "Estou com problemas para renegociar meu empréstimo estudantil.",
    "caso_hipoteca": "O banco está me cobrando taxas abusivas na minha hipoteca.",
    "caso_conta": "Minha conta foi encerrada sem aviso e perdi o acesso ao meu saldo.",
    "caso_roubo_1": "Alguém fez compras no meu nome, quero reportar uma fraude.",
    "caso_roubo_2": "Tem transações estranhas no meu extrato, acho que fui vítima de golpe.",
    "caso_outros_1": "Quero atualizar meu endereço de correspondência.",
    "caso_outros_2": "Estou tentando mudar minha senha, mas o sistema não deixa.",
}

for nome, texto in casos_teste.items():
    categoria = prever_texto(texto)
    print(f"{nome}: {categoria}")